# To generate the table

## Import modules

In [9]:
try:
    print(x)

except:
    x = 10
    import pandas as pd
    import numpy as np
    import json
    from rich import print
    

10

## Read exp IDs
Read the table with experiment IDs and pos

In [10]:
# Dataframe which stores the detail of the experiment
# details = {ID, x_pos, z_pos, ori}
exp_df = pd.read_csv("exp_details.csv")

exp_id_modified = [] # Need to add two '

for idx, id in enumerate(exp_df['Wedge_name']):
    exp_id_modified.append("'" + str(id) + "'")

# adding to data frame
exp_df['Wedge_name'] = exp_id_modified

## Generate parameter statistics

In [11]:
# The brush parameter data. These parameters will be varie to get the sensitivity
brush_parameter_mean_SD = {
    'young':{
        'mean':2.7e6,
        'SD':1.1e6
    },
    'brush_friction':{
        'mean':44,
        'SD':2
    },
    'print_frction':{
        'mean':24,
        'SD':2
    },
    'radius_diff':{
        'mean':0.0,
        'SD':0.1e-3
    }
}

## Generate sensitivity variation

In [12]:
number_variation    = 17 # has to be odd so that mean is a the center
positive_variations = (number_variation - 1)/2

for key, val in brush_parameter_mean_SD.items():
    sd   = val['SD']
    mean = val['mean'] 

    spacing   = np.linspace(0, sd, positive_variations + 1)
    variation = set(list(mean + spacing) + list(mean - spacing))
    variation.remove(mean)
    variation = list(variation)
    variation.sort()

    brush_parameter_mean_SD[key]["variation"] = variation


## Generate sensitivity table
Generate the whole table for the sensitivity simulation

In [13]:
sensitivity_table_df = pd.DataFrame()

# loop though all experiments
for idx, exp in exp_df.iterrows():
    # copy vales from exp_df 
    exp_dict_def = dict(exp)

    # write the default value
    for key, val in brush_parameter_mean_SD.items():
        exp_dict_def[key] = val['mean']

    sensitivity_table_df = sensitivity_table_df.append(exp_dict_def, ignore_index = True)
    
    # loop through all the parameters that are varied
    for key, val in brush_parameter_mean_SD.items(): 
        exp_dict = exp_dict_def

        # loop through all values of the parameter to change the correct default value
        for var_value in brush_parameter_mean_SD[key]["variation"]:            
                 
            for def_key, def_val in brush_parameter_mean_SD.items():
                # Assign the default value  which will be changed next
                exp_dict[def_key] = def_val['mean']
                exp_dict[key] = var_value
            
            sensitivity_table_df = sensitivity_table_df.append(exp_dict, ignore_index = True)

## Converting data frame into a list of string
Each member of a string is a line in the .table file

In [14]:
# getting the column names
col_names = list(sensitivity_table_df.keys())

col_names_str = "ID"
for name in col_names:
    col_names_str = col_names_str + " " + name

sensitivity_table_string = []

# Looping through all rows
for idx, row in sensitivity_table_df.iterrows():
    sim_str = str(idx)

    for name in col_names:
        sim_str = sim_str + " " + str(row[name])

    sensitivity_table_string.append(sim_str)

## Split the table
Split the table so that it can be run in mutliple machines

In [15]:
from math import *

number_cores = 4
hours = 8
run_per_hrs       = 1
number_run_per_VM = number_cores * hours * run_per_hrs
number_runs = len(sensitivity_table_string)
number_VMs  = ceil(number_runs / number_run_per_VM)

run_id_idx = 0
runs_left  = number_runs

# Writing many parameter table each with a limited number
# of rows
for VM in range(number_VMs):
    with open('VM_ID_' + str(VM) + '_parametric.table', "w") as file:

        # writing the column names
        file.write(col_names_str)
        file.write("\n")


        no_runs_in_VM = number_run_per_VM if runs_left > number_run_per_VM else runs_left
        for run in range(no_runs_in_VM):
            file.write(sensitivity_table_string[run_id_idx])
            file.write("\n")
            run_id_idx += 1


        runs_left = runs_left - number_run_per_VM

with open('parameter.table', 'w') as file:
    # Writing the column names
    file.write(col_names_str)
    file.write("\n")

    # Writing each rows
    for string in sensitivity_table_string:
         file.write(string)
         file.write("\n")

## Predicted number of sims

In [16]:
number_exps        = len(exp_df)
number_varied_ppts = len(brush_parameter_mean_SD)

number_predicted   = number_exps * (1 + (number_variation - 1)*number_varied_ppts)
print(number_predicted - len(sensitivity_table_string))

0